In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, RandomSampler


In [2]:
def get_file_name(x):
    return os.path.splitext(os.path.split(x)[-1])[0]


In [3]:
path = '000-000'
file_paths = [os.path.join(r, f) for r, _, fs in os.walk(path) for f in fs]
file_paths.sort()
img_paths, mat_paths = list(filter(lambda x: x.endswith('.png'), file_paths)), list(filter(lambda x: x.endswith('.npy'), file_paths))
display(len(file_paths), file_paths[:3])
display(len(img_paths), img_paths[:3])
display(len(mat_paths), mat_paths[:3])
assert all((get_file_name(x) == get_file_name(y) for x, y in zip(img_paths, mat_paths)))


120000

['000-000\\000074a334c541878360457c672b6c2e\\012.npy',
 '000-000\\000074a334c541878360457c672b6c2e\\012.png',
 '000-000\\000074a334c541878360457c672b6c2e\\013.npy']

60000

['000-000\\000074a334c541878360457c672b6c2e\\012.png',
 '000-000\\000074a334c541878360457c672b6c2e\\013.png',
 '000-000\\000074a334c541878360457c672b6c2e\\014.png']

60000

['000-000\\000074a334c541878360457c672b6c2e\\012.npy',
 '000-000\\000074a334c541878360457c672b6c2e\\013.npy',
 '000-000\\000074a334c541878360457c672b6c2e\\014.npy']

In [4]:
T = np.load(mat_paths[0])
T


array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -1.34358856e-07,  1.00000012e+00,
        -2.68717713e-07],
       [ 0.00000000e+00, -1.00000012e+00, -1.34358856e-07,
        -2.00000024e+00]])

In [5]:
def invert_transform(T):
    R, t = T[:3, :3], T[:3, 3:]
    R2 = R.T
    t2 = - R2 @ t
    return np.concatenate([np.concatenate([R2, t2], axis=1), np.array([[0, 0, 0, 1]])], axis=0)

invert_transform(T) @ np.concatenate([T, np.array([[0, 0, 0, 1]])], axis=0)


array([[1.        , 0.        , 0.        , 0.        ],
       [0.        , 1.00000024, 0.        , 0.        ],
       [0.        , 0.        , 1.00000024, 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [ ]:
class NVSDataset(Dataset):
    def __getitem__(self, i):
        